# Phase A: The MLE Approach

Here, we use the MLE approach to detect the likelihood of getting a word in a spam or ham message using [the SMS spam dataset collection](https://archive.ics.uci.edu/dataset/228/sms+spam+collection).

## Limitations
Vocabulary Bias: By using .split(), every unique string is treated (including punctuation attached to words like "win!") as a unique feature.

Bag-of-Words Assumption: This model assumes that the order of words doesn't matter, only their frequency.

Zero-multiplier: If a sentence containing many spam words has a single word which is not in our dataset of spam words, because
we multiply probabilities, it will render the probability of the whole message "0", even though the presence of multiple spam words
makes it very likely that it is spam.

In [12]:
import pandas as pd
from collections import defaultdict
import os

# Get the path relative to the notebook location
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
data_path = os.path.join('..', 'data', 'sms_spam', 'SMSSpamCollection')

# Load the training data
df = pd.read_csv(data_path, sep='\t', header=None, names=['label', 'message'])

# Separate spam and ham messages
spam_messages = df[df['label'] == 'spam']['message']
ham_messages = df[df['label'] == 'ham']['message']

# Count word occurrences per category
def count_words_by_category(messages):
    word_counts = defaultdict(int)
    for message in messages:
        words = message.lower().split()
        for word in words:
            word_counts[word] += 1
    return word_counts

spam_word_counts = count_words_by_category(spam_messages)
ham_word_counts = count_words_by_category(ham_messages)

# Calculate theta_word for each category
total_spam_words = sum(spam_word_counts.values())
total_ham_words = sum(ham_word_counts.values())

theta_spam = {word: count / total_spam_words for word, count in spam_word_counts.items()}
theta_ham = {word: count / total_ham_words for word, count in ham_word_counts.items()}

print(theta_ham, theta_spam)


{'go': 0.003446927455211667, 'until': 0.0003186235462800701, 'jurong': 1.4482888467275914e-05, 'point,': 1.4482888467275914e-05, 'crazy..': 1.4482888467275914e-05, 'available': 0.00015931177314003505, 'only': 0.0017089808391385578, 'in': 0.011441481889147972, 'bugis': 7.241444233637956e-05, 'n': 0.0019841557200168, 'great': 0.0011151824119802454, 'world': 0.000333106434747346, 'la': 2.8965776934551828e-05, 'e': 0.0011296653004475212, 'buffet...': 1.4482888467275914e-05, 'cine': 8.689733080365548e-05, 'there': 0.0018103610584094892, 'got': 0.0033020985705389085, 'amore': 1.4482888467275914e-05, 'wat...': 0.00017379466160731097, 'ok': 0.0022593306008950424, 'lar...': 0.00026069199241096646, 'joking': 4.344866540182774e-05, 'wif': 0.00039103798861644967, 'u': 0.01275942473967008, 'oni...': 2.8965776934551828e-05, 'dun': 0.0007965588657001752, 'say': 0.001158631077382073, 'so': 0.005778672498443089, 'early': 0.0003620722116818978, 'hor...': 2.8965776934551828e-05, 'c': 0.000825524642634727